In [1]:
import pandas as pd

In [2]:
qb24actual = pd.read_csv('../data/2024/qb_actual24.csv')
rb24actual = pd.read_csv('../data/2024/rb_actual24.csv')
wr24actual = pd.read_csv('../data/2024/wr_actual24.csv')
te24actual = pd.read_csv('../data/2024/te_actual24.csv')
k24actual = pd.read_csv('../data/2024/k_actual24.csv')
dst24actual = pd.read_csv('../data/2024/dst_actual24.csv')

qb24projections = pd.read_csv('../data/2024/qb_projections24.csv')
rb24projections = pd.read_csv('../data/2024/rb_projections24.csv')
wr24projections = pd.read_csv('../data/2024/wr_projections24.csv')
te24projections = pd.read_csv('../data/2024/te_projections24.csv')
k24projections = pd.read_csv('../data/2024/k_projections24.csv')
dst24projections = pd.read_csv('../data/2024/dst_projections24.csv')

In [3]:
qb24actual.head()

,Rank,Player,CMP(Passing),ATT(Passing),PCT(Passing),YDS(Passing),Y/A(Passing),TD(Passing),INT,SACKS,ATT(Rushing),YDS(Rushing),TD(Rushing),FL,G,FPTS,FPTS/G,ROST
0,1.0,Lamar Jackson (BAL),316.0,474.0,66.7,"4,172",8.8,41.0,4.0,23.0,139.0,915.0,4.0,5.0,17.0,434.4,25.6,98.9%
1,2.0,Josh Allen (BUF),307.0,483.0,63.6,"3,731",7.7,28.0,6.0,14.0,102.0,531.0,12.0,2.0,17.0,385.1,22.7,99.2%
2,3.0,Joe Burrow (CIN),460.0,652.0,70.6,"4,918",7.5,43.0,9.0,48.0,42.0,201.0,2.0,5.0,17.0,381.9,22.5,95.3%
3,4.0,Baker Mayfield (TB),407.0,570.0,71.4,"4,500",7.9,41.0,16.0,40.0,60.0,378.0,3.0,2.0,17.0,381.8,22.5,89.8%
4,5.0,Jayden Daniels (WAS),331.0,480.0,69.0,"3,568",7.4,25.0,9.0,47.0,148.0,891.0,6.0,0.0,17.0,364.7,21.5,96.5%


In [4]:
qb24projections.head()

,Player,Team,ATT(Passing),CMP(Passing),YDS(Passing),TD(Passing),INTS,ATT(Rushing),YDS(Rushing),TD(Rushing),FL,FPTS
0,Josh Allen,BUF,564.9,368.1,"4,062.4",27.7,14.0,108.7,558.7,8.9,4.0,360.5
1,Jalen Hurts,PHI,520.2,339.4,"3,810.6",24.4,12.1,144.1,619.8,11.2,3.8,359.9
2,Lamar Jackson,BAL,463.7,303.5,"3,567.9",25.0,10.6,146.0,836.0,5.4,4.4,339.2
3,Patrick Mahomes II,KC,588.1,394.7,"4,391.7",32.7,11.2,72.0,376.5,1.8,3.1,337.7
4,Anthony Richardson Sr.,IND,512.2,325.9,"3,506.3",20.0,11.4,128.1,648.3,7.9,5.0,311.0


In [5]:
def remove_symbols(df):
    return df.replace([',', '%'], '', regex=True)

def split_player_team(df):
    if 'Team' not in df.columns and 'Player' in df.columns and 'SAFETY' not in df.columns:
        df[['Player', 'Team']] = df['Player'].str.extract(r'^(.*) \((.*)\)$')
    return df

def rename_columns(df, keyword):
    df.columns = [f"{keyword}{col}" if col not in ['Player', 'Team'] else col for col in df.columns]
    return df

def merge_dataframes(df1, df2):
    if df1.empty:
        return df2
    if df2.empty:
        return df1
    if 'Team' in df1.columns or 'Team' not in df2.columns:
        return pd.merge(df1, df2, on=['Player'], how='outer')
    else:
       return pd.merge(df1, df2, on=['Player', 'Team'], how='outer')

In [6]:
dataframes = [
    qb24actual, rb24actual, wr24actual, te24actual, k24actual, dst24actual,
    qb24projections, rb24projections, wr24projections, te24projections, k24projections, dst24projections
]

dataframes = [remove_symbols(df) for df in dataframes]
dataframes = [split_player_team(df) for df in dataframes]

for i in range(0, len(dataframes)):
    if i < len(dataframes)/2:
        dataframes[i] = rename_columns(dataframes[i], 'actual_')
    else:
        dataframes[i] = rename_columns(dataframes[i], 'projections_')

(qb24actual, rb24actual, wr24actual, te24actual, k24actual, dst24actual,
 qb24projections, rb24projections, wr24projections, te24projections, k24projections, dst24projections) = dataframes


In [7]:
qb24full = merge_dataframes(qb24actual, qb24projections).dropna()
rb24full = merge_dataframes(rb24actual, rb24projections).dropna()
wr24full = merge_dataframes(wr24actual, wr24projections).dropna()
te24full = merge_dataframes(te24actual, te24projections).dropna()
k24full = merge_dataframes(k24actual, k24projections).dropna()
dst24full = merge_dataframes(dst24actual, dst24projections).dropna()

In [8]:
qb24full.drop(columns=['Team_y'], inplace=True)
qb24full.rename(columns={'Team_x': 'Team'}, inplace=True)

rb24full.drop(columns=['Team_y'], inplace=True)
rb24full.rename(columns={'Team_x': 'Team'}, inplace=True)

wr24full.drop(columns=['Team_y'], inplace=True)
wr24full.rename(columns={'Team_x': 'Team'}, inplace=True)

te24full.drop(columns=['Team_y'], inplace=True)
te24full.rename(columns={'Team_x': 'Team'}, inplace=True)

k24full.drop(columns=['Team_y'], inplace=True)
k24full.rename(columns={'Team_x': 'Team'}, inplace=True)

In [9]:
dst24full = dst24full[['Player', 'Team', 'actual_Rank'] + [col for col in dst24full.columns if col not in ['Player', 'Team', 'actual_Rank']]]
qb24full = qb24full[['Player', 'Team', 'actual_Rank'] + [col for col in qb24full.columns if col not in ['Player', 'Team', 'actual_Rank']]]
rb24full = rb24full[['Player', 'Team', 'actual_Rank'] + [col for col in rb24full.columns if col not in ['Player', 'Team', 'actual_Rank']]]
wr24full = wr24full[['Player', 'Team', 'actual_Rank'] + [col for col in wr24full.columns if col not in ['Player', 'Team', 'actual_Rank']]]
te24full = te24full[['Player', 'Team', 'actual_Rank'] + [col for col in te24full.columns if col not in ['Player', 'Team', 'actual_Rank']]]
k24full = k24full[['Player', 'Team', 'actual_Rank'] + [col for col in k24full.columns if col not in ['Player', 'Team', 'actual_Rank']]]

In [11]:
dst24full = dst24full.sort_values(by='actual_Rank')
qb24full = qb24full.sort_values(by='actual_Rank')
rb24full = rb24full.sort_values(by='actual_Rank')
wr24full = wr24full.sort_values(by='actual_Rank')
te24full = te24full.sort_values(by='actual_Rank')
k24full = k24full.sort_values(by='actual_Rank')

In [13]:
qb24full.to_csv('../data/qb24full.csv', index=False)
rb24full.to_csv('../data/rb24full.csv', index=False)
wr24full.to_csv('../data/wr24full.csv', index=False)
te24full.to_csv('../data/te24full.csv', index=False)
k24full.to_csv('../data/k24full.csv', index=False)
dst24full.to_csv('../data/dst24full.csv', index=False)